In [4]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [4]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [5]:
from schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
    TestSchema,
)

In [6]:
from typing import Any


async def _dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [7]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            rel_map=TableMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=[User.name],
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=_dummy_loader,
                    push={User.name, User.age},
                    match_by=[User.name],
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                        "organization_countries": Organization.countries,
                    },
                    match_by=[Organization.name, Organization.city],
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [8]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "organization_countries": ["USA", "Mexico", "Canada"],
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "organization_countries": ["USA", "Mexico"],
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "organization_countries": ["USA", "Canada"],
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [9]:
from pathlib import Path
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db = DataBase(
      schema=TestSchema,
      # url=Path("./test.xlsx")
    )
    rec = await data_source.load([data], db=db)

    stats = pstats.Stats(pr)

Loading: `Search`: 100%|██████████| 1/1 [00:00<00:00, 10.69it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\lworm\AppData\Local\Temp\ipykernel_14044\4112843260.py:11 in <module>                   │
│                                                                                                  │
│    8 │     schema=TestSchema,                                                                    │
│    9 │     # url=Path("./test.xlsx")                                                             │
│   10 │   )                                                                                       │
│ ❱ 11 │   rec = await data_source.load([data], db=db)                                             │
│   12 │                                                                                           │
│   13 │   stats = pstats.Stats(pr)                                                                │
│   14                                                                                             │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:880 in load                   │
│                                                                                                  │
│   877 │   │   db = db if db is not None else DataBase()                                          │
│   878 │   │   in_data: InData = {((), ()): dat for dat in data}                                  │
│   879 │   │   rest_data: RestData = []                                                           │
│ ❱ 880 │   │   loaded = await _load_records(db[self.record_type], self, in_data, rest_data)       │
│   881 │   │                                                                                      │
│   882 │   │   # Perform second pass to load remaining data                                       │
│   883 │   │   for table_map, tree_data in rest_data:                                             │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:750 in _load_records          │
│                                                                                                  │
│   747 │   │   records[(rec_idx, parent_idx, path_idx)] = rec                                     │
│   748 │                                                                                          │
│   749 │   # Upload main records into the database.                                               │
│ ❱ 750 │   table |= {idx: rec for (idx, _, _), rec in records.items()}                            │
│   751 │                                                                                          │
│   752 │   if len(rest_records) > 0:                                                              │
│   753 │   │   # Collect remaining data for later loading.                                        │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2772 in __ior__                 │
│                                                                                                  │
│   2769 │   │   other: Data[Any, Any, Any, Any, Any, DynBackendID] | Input[Any, Any],             │
│   2770 │   ) -> Data[ValT, IdxT, CrudT, RelT, CtxT, BaseT]:                                      │
│   2771 │   │   """Upserting assignment."""                                                       │
│ ❱ 2772 │   │   self._mutate(other, mode="upsert")                                                │
│   2773 │   │   return cast(                                                                      │
│   2774 │   │   │   Data[ValT, IdxT, CrudT, RelT, CtxT, BaseT],                                   │
│   2775 │   │   │   self,                                   

In [8]:
import polars as pl
pl.read_database(
                'SELECT * FROM schema_Organization_countries', db.engine
            )

_from__id,idx,value
str,i64,str
"""294904d0-e719-4599-899a-30bb20…",0,"""USA"""
"""294904d0-e719-4599-899a-30bb20…",1,"""Mexico"""
"""294904d0-e719-4599-899a-30bb20…",2,"""Canada"""
"""65d054da-b302-4728-aac0-8a1698…",0,"""USA"""
"""65d054da-b302-4728-aac0-8a1698…",1,"""Canada"""
"""5f96952f-62aa-46da-b2e4-842a90…",0,"""USA"""
"""5f96952f-62aa-46da-b2e4-842a90…",1,"""Mexico"""


In [9]:
from pathlib import Path
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db2 = DataBase(
      schema=TestSchema,
      # url=Path("./test.xlsx")
    )
    rec = await data_source.load([data], db=db2)

    stats = pstats.Stats(pr)

Post-loading: `Assignment`: 100%|██████████| 6/6 [00:00<00:00, 159.93it/s]


In [10]:
# db[User].extract()

In [11]:
db[User].keys()[0]

'1f90a58f-a520-4527-9c52-07f25dd33bad'

In [12]:
del db[User][db[User].keys()[0]]

In [13]:
db[User].df()

_id,name,age
str,str,i64
"""4f4f0578-e7d4-4d92-882c-73f02d…","""John""",31
"""dc671c32-892c-4008-af31-9ada14…","""Jane""",56


In [14]:
db2[User].df()

_id,name,age
str,str,i64
"""16c6b407-45ff-4d35-b12b-6cf1d0…","""John""",31
"""7b540138-97c2-43ac-ab0d-578171…","""Jane""",56
"""8b21ee7b-85ec-41db-91a7-bc86b4…","""Jack""",66


In [15]:
db3 = db | db2
db3.describe()


{
    'contents': {
        'records': {
            'schema.Project': 3,
            'schema.Organization': 6,
            'schema.Task': 12,
            'schema.User': 5,
            'schema.Search': 1
        },
        'arrays': {'schema.Organization.countries': 14},
        'relations': {'schema.SearchResult': 3, 'schema.Assignment': 12, 'schema.Membership': 14}
    },
    'backend': None,
    'overlay': 'upsert/(a7d8c35b1d|0813e81f13)/c9ba0523'
}

In [16]:
db4 = db << db2
db4.describe()


{
    'contents': {
        'records': {
            'schema.Project': 3,
            'schema.Organization': 6,
            'schema.Task': 12,
            'schema.User': 5,
            'schema.Search': 1
        },
        'arrays': {'schema.Organization.countries': 14},
        'relations': {'schema.SearchResult': 3, 'schema.Assignment': 12, 'schema.Membership': 14}
    },
    'backend': None,
    'overlay': 'insert/(a7d8c35b1d<<0813e81f13)/a244af2f'
}

In [17]:
db5 = db >> db2
db5.describe()


{
    'contents': {
        'records': {
            'schema.Project': 3,
            'schema.Organization': 3,
            'schema.Task': 6,
            'schema.User': 2,
            'schema.Search': 1
        },
        'arrays': {'schema.Organization.countries': 7},
        'relations': {'schema.SearchResult': 3, 'schema.Assignment': 6, 'schema.Membership': 7}
    },
    'backend': None,
    'overlay': 'update/(a7d8c35b1d>>0813e81f13)/6d2fe804'
}

In [18]:
x = db[Search.results]._link
assert x is not None
x.df()

schema.Search.term,schema.Search.results._rel._id,number,name,start,end,status,org__id
str,str,i64,str,date,date,str,str
"""test""","""104d65b581""",2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""65d054da-b302-4728-aac0-8a1698…"
"""test""","""11faab1a21""",1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""5f96952f-62aa-46da-b2e4-842a90…"
"""test""","""14941c52d3""",0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""294904d0-e719-4599-899a-30bb20…"


In [19]:
x = db[Search.results.x.tasks.x.assignees]
print(x.select_str)
x.keys()
x.df(sort_by=[User.name])

SELECT DISTINCT "schema.Search".term AS "schema.Search.term",
                "schema.Search.results".number AS "schema.Search.results.number",
                "schema.Search.results.tasks"._id AS "schema.Search.results.tasks._id",
                "schema.Task.assignees"._id AS "schema.Search.results.tasks.assignees._id",
                "schema.Task.assignees"._id AS _id,
                "schema.Task.assignees".name AS name,
                "schema.Task.assignees".age AS age
FROM "schema_Search" AS "schema.Search"
JOIN "schema_SearchResult" AS "schema.Search.results._rel" ON "schema.Search.results._rel"._from_term = "schema.Search".term
JOIN "schema_Project" AS "schema.Search.results" ON "schema.Search.results._rel"._to_number = "schema.Search.results".number
JOIN "schema_Task" AS "schema.Search.results.tasks" ON "schema.Search.results.tasks".project_number = "schema.Search.results".number
JOIN "schema_Assignment" AS "schema.Search.results.tasks.assignees._rel" ON "schema.Search.resul

schema.Search.term,schema.Search.results.number,schema.Search.results.tasks._id,_id,name,age
str,i64,str,str,str,i64
"""test""",0,"""6fa3b903-4047-4705-9ea1-72ffc6…","""dc671c32-892c-4008-af31-9ada14…","""Jane""",56
"""test""",1,"""62a4bd91-0fcb-4b9d-b601-8cc4c3…","""dc671c32-892c-4008-af31-9ada14…","""Jane""",56
"""test""",2,"""c93062b7-f2a6-4117-931f-78577a…","""4f4f0578-e7d4-4d92-882c-73f02d…","""John""",31
"""test""",0,"""1f2c57b8-fc3a-4693-8fcb-b3548b…","""4f4f0578-e7d4-4d92-882c-73f02d…","""John""",31
"""test""",1,"""c64fd57f-2e02-4c2a-acf1-05192f…","""4f4f0578-e7d4-4d92-882c-73f02d…","""John""",31
"""test""",0,"""95584452-de41-4f64-af8f-f0dfdc…","""4f4f0578-e7d4-4d92-882c-73f02d…","""John""",31


In [20]:
db.describe()


{
    'schema': {
        'package': 'py-research',
        'module': 'schema',
        'object': 'TestSchema',
        'package_version': '^5.1.1.dev152+g2c0bcb5.d20250123',
        'repo': 'https://github.com/cloudlane-one/py-research.git',
        'repo_revision': '3783bc57d23333ab436eb55cec0be4a6abad92a4',
        'module_path': WindowsPath('exp/schema.py'),
        'module_dirty': False,
        'docs_url': 'https://cloudlane-one.github.io/py-research/'
    },
    'contents': {
        'records': {
            'schema.Project': 3,
            'schema.Organization': 3,
            'schema.Task': 6,
            'schema.User': 2,
            'schema.Search': 1
        },
        'arrays': {'schema.Organization.countries': 7},
        'relations': {'schema.SearchResult': 3, 'schema.Assignment': 6, 'schema.Membership': 7}
    },
    'backend': None
}

In [21]:
nodes, edges = db.to_graph([Search, Project._table[Project.number == 1], Organization])

In [23]:
nodes

,node_id,table,schema.Search.term,schema.Search.result_count,schema.Project.number,schema.Project.name,schema.Project.start,schema.Project.end,schema.Project.status,schema.Project.org__id,schema.Organization._id,schema.Organization.name,schema.Organization.address,schema.Organization.city
0,0,schema_Search,test,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,schema_Project,NaN,NaN,1.0,cleaning shoes,2020-01-02,2020-01-05,done,5f96952f-62aa-46da-b2e4-842a90a22c42,NaN,NaN,NaN,NaN
2,2,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,294904d0-e719-4599-899a-30bb20325890,Bakery,Main Street 1,Bakerville
3,3,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5f96952f-62aa-46da-b2e4-842a90a22c42,Shoe Shop,Main Street 2,Shoetown
4,4,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65d054da-b302-4728-aac0-8a16983b59e8,Car Shop,Main Street 3,Cartown


In [24]:
edges

,source,target,ltr,rtl,schema.SearchResult.score,schema.SearchResult._to_number,schema.SearchResult._from_term,schema.SearchResult._id
0,1,3,org__id,None,NaN,NaN,NaN,NaN
1,0,1,_to_number,_from_term,0.8,1.0,test,11faab1a21


In [25]:
db[User].values()


[
    User({'_id': '4f4f0578-e7d4-4d92-882c-73f02d7408a3', 'name': 'John', 'age': 31}),
    User({'_id': 'dc671c32-892c-4008-af31-9ada149a4bc6', 'name': 'Jane', 'age': 56})
]

In [26]:
db[Project].df()

number,name,start,end,status,org__id
i64,str,date,date,str,str
0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""294904d0-e719-4599-899a-30bb20…"
1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""5f96952f-62aa-46da-b2e4-842a90…"
2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""65d054da-b302-4728-aac0-8a1698…"


In [27]:
db[Search.term].values()

['test']

In [28]:
list(db[Search.result_count].items())

[('test', 3)]

In [29]:
list(db[Search.results.x.org].items())


[
    (
        ('test', 0, '294904d0-e719-4599-899a-30bb20325890'),
        Organization({'_id': '294904d0-e719-4599-899a-30bb20325890', 'name': 'Bakery', 'address': 'Main Street 1', 'city': 'Bakerville'})
    ),
    (
        ('test', 1, '5f96952f-62aa-46da-b2e4-842a90a22c42'),
        Organization({'_id': '5f96952f-62aa-46da-b2e4-842a90a22c42', 'name': 'Shoe Shop', 'address': 'Main Street 2', 'city': 'Shoetown'})
    ),
    (
        ('test', 2, '65d054da-b302-4728-aac0-8a16983b59e8'),
        Organization({'_id': '65d054da-b302-4728-aac0-8a16983b59e8', 'name': 'Car Shop', 'address': 'Main Street 3', 'city': 'Cartown'})
    )
]

In [ ]:
class TestClass[*T]:
  ...
  
def valid_test_1[*T1, *T2](a: TestClass[*T1], b: TestClass[*T2]) -> tuple[*T1]:
  ...
  
x = Test[int, str]()
y = Test[int, str, float]()

z = test(y)

In [ ]:
db[Search.results].items()


[
    (
        ('test', 0),
        Project({'number': 0, 'name': 'baking cake', 'start': datetime.date(2020, 1, 1), 'end': datetime.date(2020, 1, 4), 'status': 'done'})
    ),
    (
        ('test', 1),
        Project({'number': 1, 'name': 'cleaning shoes', 'start': datetime.date(2020, 1, 2), 'end': datetime.date(2020, 1, 5), 'status': 'done'})
    ),
    (
        ('test', 2),
        Project({'number': 2, 'name': 'fixing cars', 'start': datetime.date(2020, 1, 3), 'end': datetime.date(2020, 1, 6), 'status': 'started'})
    )
]

In [ ]:
class X[T: list, U: tuple, V: dict]:
  def __init__[A, B, C](self: X[list[A], tuple[A], dict[B, C]], a: A, bc: tuple[B, C]) -> None:
    ...
    
x = X[list[int], tuple[int], dict[float, str]](1, (1, "0"))

In [2]:
from typing import Generic, TypeVar

T = TypeVar('T', covariant=True, bound="Read")
U = TypeVar('U', covariant=True, bound="Contra")
V = TypeVar('V', contravariant=True)

class Read:
  ...
  
class ReadWrite(Read):
  ...
  
class Contra(Generic[V]):
  ...
  
class Container(Generic[T, U]):
  
  def test[T2: Read](self: Container[T2, Any], x: Container[]) -> T2:
    ...

x = Container[Read]()
y = Container[ReadWrite]()

z = x.test(y)

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│   def test[T2: Read](self: Container[T2, Any], x: Container[]) -> T2:                            │
│                                                   ▲                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax. Perhaps you forgot a comma?

In [ ]:
IV = TypeVar("IV", covariant=True)

class Idx(Generic[IV]):
  ...
  
def merge[T, U](a: T, b: U) -> T | U:
  ...
  
def extract[T](a: Idx[T]) -> T:
  ...
  
x = Idx[int]()
y = Idx[str]()

z = merge(x, y)
w = extract(z)

In [ ]:
from typing import Literal, overload, TypeVar, reveal_type
from typing_extensions import TypeForm

T = TypeVar("T")
U = TypeVar("U")


@overload
def split_type_typeform(x: type) -> Literal["type"]: ...


@overload
def split_type_typeform(x: TypeForm) -> Literal["typeform"]: ...


def split_type_typeform(x) -> Any: ...


reveal_type(split_type_typeform(int))  # Literal["type"], correct
reveal_type(split_type_typeform(list))  # Literal["type"], correct
reveal_type(
    split_type_typeform(int | list)  # Literal["type"], but should be Literal["typeform"]
)
reveal_type(
    split_type_typeform(list[int])  # Literal["type"], but should be Literal["typeform"]
)


def only_accept_type(x: type) -> None: ...


only_accept_type(int)  # ok
only_accept_type(list[int])  # ok (side note: should this be rejected?)
only_accept_type(int | list)  # correctly rejected

In [ ]:
from typing import TypeVar, Union

T = TypeVar("T")
U = TypeVar("U")

class IndexedUnion(Union, Generic[T]):
  ...

class Meta(type):
    def __or__(cls: type[T], other: type[U]) -> IndexedUnion[T | U]:
        raise NotImplementedError()

class XX(metaclass=Meta):
    ...
    
class YY(metaclass=Meta):
    ...
    
reveal_type(XX | YY)  # Meta[Union[int, str]], correct

In [ ]:
def asdf[T](x: type[T]) -> T:
  ...
  
asdf(int)
asdf(int | str)
asdf(list[int])

In [ ]:
from typing import TypeVar, Generic, get

T = TypeVar("T")

class WWW(Generic[T]):
  pass



In [ ]:
from dataclasses import dataclass
from typing import Callable, Protocol, ParamSpec, Concatenate

P = ParamSpec("P")

class SingleArgCallable(Protocol[T]):
  def __call__(self, arg: T, **kwargs: Any) -> Any:
    ...

@dataclass
class KwClass:
  a: int
  b: str = "default"

def accept_int_callable(x: Callable[Concatenate[int, P], T]) -> Callable[P, T]:
  ...

x = accept_int_callable(KwClass)

In [13]:
from typing import reveal_type

class TestMetaType[T]:
  ...

class TestType[T](type, TestMetaType[int]):
  ...
  
def do_sth[T](x: TestMetaType[T]) -> T:
  ...
  
class X[T](metaclass=TestType[T]):
  ...
  
x = do_sth(X)
reveal_type(x)

In [ ]:
from dataclasses import dataclass, field

@dataclass(kw_only=True)
class DC:
  a: int
  b: str = field(default="default")
  
@dataclass(kw_only=True)
class DC2(DC):
  a = field(default=1)
  c: float

DC2()

In [14]:
from typing import TypeVar, Generic

T = TypeVar("T", contravariant=True)
U = TypeVar("U", covariant=True)

class Test(Generic[T]):
  ...
  
class Test2(Generic[U]):
  ...
  
def test[V](x: Test[V], y: Test2[V]) -> V:
  ...
  
yyyy = test(Test[float](), Test2[int]())
reveal_type(yyyy)


In [11]:
import sqlalchemy as sqla

class Test(sqla.SQLColumnExpression):
  def __clause_element__(self) -> sqla.ColumnElement:
    return sqla.column("test")
  
Test() > 1

str(sqla.select(Test()).select_from(sqla.table("tab", sqla.column("test"))))

'SELECT test \nFROM tab'

In [6]:
WWW[int]().__orig_class__

__main__.WWW[int]

In [ ]:
class A:
  ...
  
class B(A):
  ...
  
class C(A):
  ...
  
class D(B):
  ...
  
def teest[T](x: Contra[T], y: T) -> T:
  ...
  
z = teest(Contra[B](), D())

In [ ]:
class Test[*T]:
  ...
  
def test[*T, *U](a: Test[*T], b: Test[*T, *U]) -> Test[*U]:
  ...
  
x = Test[int, str, float]()
y = Test[int, str]()

z = test(y, x)

In [ ]:
list(db[Search.results]._abs_cols.keys())

['number', 'name', 'start', 'end', 'status', 'org__id']

In [32]:
db[Search.results].df()

schema.Search.term,number,name,start,end,status,org__id
str,i64,str,date,date,str,str
"""test""",0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""294904d0-e719-4599-899a-30bb20…"
"""test""",1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""5f96952f-62aa-46da-b2e4-842a90…"
"""test""",2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""65d054da-b302-4728-aac0-8a1698…"


In [33]:
db[Search.results.x.org].df()

schema.Search.term,schema.Search.results.number,_id,name,address,city
str,i64,str,str,str,str
"""test""",0,"""294904d0-e719-4599-899a-30bb20…","""Bakery""","""Main Street 1""","""Bakerville"""
"""test""",1,"""5f96952f-62aa-46da-b2e4-842a90…","""Shoe Shop""","""Main Street 2""","""Shoetown"""
"""test""",2,"""65d054da-b302-4728-aac0-8a1698…","""Car Shop""","""Main Street 3""","""Cartown"""


In [34]:
db[Organization].df()

_id,name,address,city
str,str,str,str
"""294904d0-e719-4599-899a-30bb20…","""Bakery""","""Main Street 1""","""Bakerville"""
"""5f96952f-62aa-46da-b2e4-842a90…","""Shoe Shop""","""Main Street 2""","""Shoetown"""
"""65d054da-b302-4728-aac0-8a1698…","""Car Shop""","""Main Street 3""","""Cartown"""


In [35]:
db[SearchResult._table[SearchResult.score.isin(slice(0.81, 0.9))]].df()

_id,score,_from_term,_to_number
str,f64,str,i64
"""14941c52d3""",0.9,"""test""",0


In [36]:
import pandas as pd


db[Organization.countries].df(pd.DataFrame)

value
schema.Organization._id              schema.Organization.countries.idx        
294904d0-e719-4599-899a-30bb20325890 0                                     USA
                                     1                                  Mexico
                                     2                                  Canada
5f96952f-62aa-46da-b2e4-842a90a22c42 0                                     USA
                                     1                                  Mexico
65d054da-b302-4728-aac0-8a16983b59e8 0                                     USA
                                     1                                  Canada

In [2]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         169 function calls (148 primitive calls) in 0.000 seconds

   Ordered by: cumulative time
   List reduced from 21 to 4 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      5/1    0.000    0.000    0.000    0.000 C:\Users\lworm\Repos\py-research\src\py_research\reflect\types.py:302(get_typevar_map)
      4/2    0.000    0.000    0.000    0.000 C:\Users\lworm\Repos\py-research\src\py_research\reflect\types.py:343(<genexpr>)
      3/2    0.000    0.000    0.000    0.000 C:\Users\lworm\Repos\py-research\src\py_research\reflect\types.py:157(hint_to_typedef)
        2    0.000    0.000    0.000    0.000 C:\Users\lworm\Repos\py-research\src\py_research\reflect\types.py:208(<genexpr>)


